In [2]:
import pandas as pd
import re
import requests
from datetime import datetime
import time
import random

In [3]:
url = 'https://docs.google.com/spreadsheets/d/13xMEbJCrMzxJheEn09Ctt0HwIfzg8jjC/export?format=csv'
df = pd.read_csv(url)

In [4]:
df = df.drop(['Nº', 'cÂMARA OU SENADO', 'Descrição ', 'Andamento do PL ', 'Observações andamento '], axis=1)

In [5]:
DEFAULT_TIMEOUT = 15  # segundos
MAX_TRIES = 5         # tentativas totais

def http_get(url, params=None, timeout=DEFAULT_TIMEOUT, max_tries=MAX_TRIES):
    """
    GET com retry exponencial + jitter e timeouts.
    Levanta a última exceção se esgotar tentativas.
    """
    last_exc = None
    for attempt in range(1, max_tries + 1):
        try:
            resp = requests.get(
                url,
                params=params,
                headers={"User-Agent": "DELOG-PL-Monitor/1.0"},
                timeout=timeout,
            )
            # Considera 5xx e 429 como transitórios
            if resp.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"HTTP {resp.status_code}", response=resp)
            resp.raise_for_status()
            return resp
        except (requests.Timeout, requests.ConnectionError, requests.HTTPError) as e:
            last_exc = e
            if attempt == max_tries:
                break
            # backoff exponencial com jitter (base 1s): 1, 2, 4, 8...
            sleep_s = (2 ** (attempt - 1)) + random.uniform(0, 0.5)
            time.sleep(sleep_s)
    # esgotou
    raise last_exc

In [6]:
def renomear_unnamed(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renomeia colunas 'Unnamed' para o mesmo nome do último '<nº Encaminhamento>' encontrado.
    Exemplo: 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9' -> '1º Encaminhamento'
    """
    cols = list(df.columns)
    novos = {}
    grupo_atual = None

    for c in cols:
        nome = c.strip() if isinstance(c, str) else c

        # detecta "nº Encaminhamento" (1º, 2º, 3º...)
        if isinstance(nome, str) and re.match(r'^\d+º\s*Encaminhamento', nome, flags=re.I):
            grupo_atual = re.sub(r'\s+', ' ', nome).strip()
            continue

        # se for Unnamed e há um grupo em curso, renomeia para o grupo
        if grupo_atual and isinstance(nome, str) and nome.lower().startswith('unnamed'):
            novos[c] = grupo_atual
        else:
            # se vier coluna normal, limpa grupo
            if not (isinstance(nome, str) and nome.lower().startswith('unnamed')):
                grupo_atual = None

    return df.rename(columns=novos)

In [7]:
def extrair_num_ano(texto):
    """
    Extrai o número e o ano do PL a partir de uma string textual.
    Retorna (numero, ano) como strings, ou (None, None) se não encontrar.
    """
    if not isinstance(texto, str):
        return None, None
    
    # Expressão regular que pega formatos como:
    # "PL nº 4603/2023", "PLnº2633/2020", "PL nº 2789, de 2021", etc.
    padrao = re.search(r'(\d{1,4}(?:\.\d{3})?)[^\d]*(?:\/|, de )\s*(\d{4})', texto)
    if padrao:
        numero = padrao.group(1).replace('.', '')  # Remove ponto se existir (ex: 1.498 → 1498)
        ano = padrao.group(2)
        return numero, ano
    return None, None

In [8]:
df[["numero_pl", "ano_pl"]] = df["Projeto de LEI "].apply(lambda x: pd.Series(extrair_num_ano(x)))

In [9]:
def buscar_info_pl(numero, ano):
    # 1) Buscar ID da proposição
    url_busca = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
    params = {"siglaTipo": "PL", "numero": numero, "ano": ano}

    try:
        res = http_get(url_busca, params=params)
        payload = res.json()
        dados = payload.get("dados", [])
        if not dados:
            return None
        proposicao = dados[0]
        id_prop = proposicao.get("id")
        if not id_prop:
            return None

        # 2) Detalhes
        det = http_get(f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id_prop}").json().get("dados", {}) or {}
        status = det.get("statusProposicao", {}) or {}
        link_inteiro_teor_pl = det.get("urlInteiroTeor") or ""
        link_ficha = det.get("uri") or f"https://www.camara.leg.br/propostas-legislativas/{id_prop}"

        # 3) Última tramitação
        data_tramit = status.get("dataHora") or ""
        sigla_orgao = status.get("siglaOrgao", "")
        situacao_tramit = status.get("descricaoSituacao", "")

        # 4) Parecer aprovado (exemplo simples; adapte se precisar de regra mais elaborada)
        parecer_data = ""
        parecer_orgao = ""
        parecer_despacho = ""
        parecer_url = ""

        try:
            tramitacoes = http_get(f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id_prop}/tramitacoes").json().get("dados", []) or []
            for t in tramitacoes:
                desp = (t.get("despacho") or "").lower()
                mov = (t.get("descricaoTramitacao") or "").lower()
                if "parecer" in desp or "parecer" in mov:
                    parecer_data = t.get("dataHora", "") or ""
                    parecer_orgao = t.get("siglaOrgao", "") or ""
                    parecer_despacho = t.get("despacho", "") or t.get("descricaoTramitacao", "")
                    # tentativa de link do parecer nos documentos da tramitação (quando existir)
                    doclist = t.get("documentos", []) or []
                    if isinstance(doclist, list) and doclist:
                        # pega o primeiro com URL
                        for d in doclist:
                            u = d.get("url", "")
                            if u:
                                parecer_url = u
                                break
                    break
        except Exception:
            # não derruba o fluxo se a busca de tramitações falhar
            pass

        return {
            "Projeto de Lei": f'{proposicao.get("siglaTipo","PL")} {proposicao.get("numero","")}/{proposicao.get("ano","")}',
            "Ementa": proposicao.get("ementa", ""),
            "Data Última Tramitação": data_tramit,
            "Órgão Última Tramitação": sigla_orgao,
            "Situação Última Tramitação": situacao_tramit,
            "Data Parecer Aprovado": parecer_data,
            "Órgão Parecer": parecer_orgao,
            "Despacho Parecer": parecer_despacho,
            "Link Inteiro Teor Parecer": parecer_url,
            "Link Inteiro Teor do PL": link_inteiro_teor_pl,
            "Link Ficha de Tramitação": link_ficha,
        }

    except Exception:
        # Falha transitória mesmo após retries: retorna None para pular o registro
        return None

In [10]:
# Buscar dados
resultados = []
for _, row in df.iterrows():
    info = buscar_info_pl(row["numero_pl"], row["ano_pl"])
    if info is not None:
        resultados.append(info)
    time.sleep(0.1)  # só para ser "educado"; o http_get já faz backoff
df_info = pd.DataFrame(resultados)

In [11]:
df_info[["numero_pl", "ano_pl"]] = df_info["Projeto de Lei"].str.extract(r'(\d+)/(\d+)')

In [12]:
df_final = pd.merge(df, df_info, on=["numero_pl", "ano_pl"], how="left")

In [13]:
df_final = df_final.drop(columns=["numero_pl", "ano_pl"])

In [14]:
df_final = renomear_unnamed(df_final)

In [15]:
# normaliza nomes: remove espaços laterais
df_final.columns = [c.strip() if isinstance(c, str) else c for c in df_final.columns]

# remove duplicata antiga, se existir
for dup in ["Projeto de LEI", "Projeto de LEI "]:
    if dup in df_final.columns:
        df_final = df_final.drop(columns=[dup])

In [20]:
df_final.to_csv("data/df_final.csv", index=False, encoding="utf-8-sig")